# No Man's Sky Updates Analysis

## Table of Contents
1. [Introduction](#Introduction)
2. [Data Description](#Data-Description)

## Introduction
The purpose of this analysis is to:
- Examine the frequency and patterns of updates for No Man's Sky.
- Provide insights into the development cycle.
- Help predict future updates.
- Identify periods of high and low activity for planning game events or releases.

## Data Description
The dataset includes:
- Dates and descriptions of each update.
- Data collected from official No Man's Sky update logs.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import calendar
import re

# Load the existing CSV data
noman_update_data = pd.read_csv('../data/Noman/noman_updates_collection.csv')

# Display the first few rows
print(noman_update_data.head())

# Display info about the dataset
print(noman_update_data.info())

  Category                                              Title  \
0    patch                 No Man's Sky Companions Patch 3.21   
1    patch  No Man's Sky Update 3.2 patch notes bring adop...   
2    patch                               Crossplay Patch 2.52   
3    patch                                Exo Mech Patch 2.42   
4    patch  No Man's Sky Beyond patch notes: VR, Multiplay...   

                  Date  
0  2021-02-19 14:45:50  
1  2021-02-17 07:40:00  
2  2020-06-12 16:25:20  
3  2020-04-16 13:50:00  
4  2019-08-15 15:57:34  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  225 non-null    object
 1   Title     225 non-null    object
 2   Date      225 non-null    object
dtypes: object(3)
memory usage: 5.4+ KB
None


In [2]:
#Bringing in Raw data due to lack of data on API.
def process_raw_data(raw_data):
    lines = raw_data.strip().split('\n')
    updates = []
    for line in lines:
        match = re.match(r'(.*?)(\d+(?:\.\d+)+)(?:\s*-?\s*(.+))?$', line.strip())
        if match:
            name, version, date_str = match.groups()
            name = name.strip()
            date = None
            if date_str:
                # Remove any leading/trailing whitespace, period, and handle "Hotfix -" prefix
                date_str = re.sub(r'^Hotfix\s*-\s*', '', date_str.strip().rstrip('.'))
                try:
                    # Try multiple date formats
                    for fmt in ('%B %d, %Y', '%B %d,%Y', '%b %d, %Y', '%b %d,%Y'):
                        try:
                            date = datetime.strptime(date_str, fmt).date()
                            break
                        except ValueError:
                            continue
                    if not date:
                        # If still no match, try a more flexible approach
                        date_parts = date_str.replace(',', '').split()
                        if len(date_parts) == 3:
                            month, day, year = date_parts
                            date = datetime(int(year), list(calendar.month_name).index(month), int(day)).date()
                except ValueError:
                    print(f"Could not parse date: {date_str}")
            updates.append({'Version': version, 'Name': name, 'Date': date})
    return pd.DataFrame(updates)

# Raw data pulled from https://www.nomanssky.com/release-log/
raw_data = """
PC Patch 1.04
PC Patch 1.05
PC Patch 1.06
PC Patch 1.07
PC Patch 1.08
PC Patch 1.09
PC Patch 1.06
Foundation Update 1.10
Patch 1.12 - December 7, 2016.
Patch 1.13 - December 12, 2016.
PathFinder Update 1.2
Path Finder Patch 1.22
Path Finder Patch 1.23
Path Finder Patch 1.24
Atlas Rises Update 1.3
Atlas Rises Patch 1.31 - August 12, 2017.
Atlas Rises Patch 1.32 - August 17, 2017.
Atlas Rises Patch 1.33 - August 22, 2017.
Atlas Rises Patch 1.34 - August 25, 2017.
Atlas Rises Patch 1.35 - September 1, 2017.
Atlas Rises Patch 1.37 - September 15, 2017.
Atlas Rises Patch 1.38 - October 3, 2017.
No Mans Sky Next 1.5
NEXT Patch 1.51 - July 26, 2018.
NEXT Patch 1.52 - July 27, 2018.
NEXT Patch 1.52.2 - July 30, 2018.
NEXT Patch 1.53 - August 1, 2018.
NEXT Patch 1.54 - August 7, 2018.
NEXT Patch 1.55 - August 10, 2018.
NEXT Patch 1.57 - August 17, 2018.
NEXT Patch 1.58 - August 30, 2018.
NEXT Patch 1.59 - September 6, 2018.
Next Patch 1.60 - September 20, 2018.
Next Patch 1.61 - September 27, 2018.
Next Patch 1.63 - October 5, 2018.
Next Patch 1.64 - October 12, 2018.
Next Patch 1.65 - October 19, 2018.
Abyss Update 1.7
Abyss Patch 1.71 - November 2, 2018.
Visions Update 1.75 
Visions Patch 1.76 - November 29, 2018.
Visions Patch 1.77 - December 13, 2018.
Beyond 2.0
Beyond Patch 2.04 - August 15, 2019.
Beyond Patch 2.05 - August 15, 2019.
Beyond Patch 2.06 - August 15, 2019.
Beyond Patch 2.07 - August 20, 2019.
Beyond Patch 2.08 - August 21, 2019.
Beyond Patch 2.09 - August 22, 2019.
Beyond Patch 2.09.1 -  August 23, 2019.
Beyond Patch 2.09.2 - August 28, 2019.
Beyond Patch 2.09.3 - August 30, 2019.
Beyond Patch 2.09.4 - September 3, 2019.
Beyond Patch 2.09.4 Hotfix - September 4, 2019.
Beyond Patch 2.11 - September 5, 2019.
Beyond Patch 2.12 - September 18, 2019.
Beyond Patch 2.13 - October 3, 2019.
Beyond Patch 2.14 - October 10, 2019.
Beyond Patch 2.15 - October 30, 2019.
Beyond Patch 2.16 - November 7, 2019.
Synthesis Update 2.2
Synthesis Patch 2.22 - November 29, 2019.
Synthesis Patch 2.23 - December 4, 2019.
Synthesis Patch 2.24 - December 16, 2019.
Synthesis Patch 2.26 - January 22, 2020.
Synthesis Patch 2.27 - February 4, 2020.
Living Ship 2.3 
Living Ship Patch 2.31 - February 21, 2020.
Living Ship Patch 2.32 - February 27, 2020.
Living Ship Patch 2.33 - March 12, 2020.
Living Ship Patch 2.34 - March 31, 2020.
Exo Mech 2.4
Exo Mech Patch 2.41 - April 8, 2020.
Exo Mech Patch 2.42 - April 16, 2020.
Cross Play 2.5 - June 10, 2020.
Crossplay Patch 2.52 - June 12, 2020.
Crossplay Patch 2.53 - June 16, 2020.
Crossplay Patch 2.54 - June 18, 2020.
Crossplay Patch 2.55 - June 26, 2020.
Desolation 2.6 
Desolation Patch 2.61 - July 22, 2020.
Desolation Patch 2.62 - August 12, 2020.
Origins 3.0
Origins Patch 3.01 - September 24, 2020.
Origins Patch 3.02 - September 29, 2020.
Origins Patch 3.03 - October 9, 2020.
Halloween Update 3.05 - October 26, 2020.
Next Generation 3.10
Next Generation Patch 3.12 - November 25, 2020.
Next Generation Patch 3.13 - November 30, 2020.
Next Generation Patch 3.15 - January 27, 2021.
Companions 3.2
Companions Patch 3.21 - February 19,2021.
Companions Patch 3.22 - February 26, 2021.
Expeditions 3.3
Expeditions Patch 3.32 - April 1, 2021.
Expeditions Patch 3.33 - April 1, 2021.
Expeditions Patch 3.34 - April 5, 2021.
Expeditions Patch 3.35 - April 7, 2021.
Expeditions Patch 3.37 - April 15, 2021.
Expeditions Patch 3.38 - April 30, 2021.
Beachhead Expedition Patch 3.40 - May 17, 2021.
Beachhead Expedition Patch 3.41 - May 21, 2021.
Beachhead Expedition Patch 3.42 - May 21, 2021.
Prisms 3.5
Prisms Patch 3.51 - June 4, 2021.
Prisms Patch 3.52 - June 10, 2021.
Prisms Patch 3.53 - June 16, 2021.
Frontiers 3.6
Frontiers Patch 3.61 - September 2, 2021.
Frontiers Patch 3.62 - September 3, 2021.
Frontiers Patch 3.63 - September 6, 2021.
Frontiers Patch 3.64 - September 8, 2021.
Frontiers Patch 3.65 - September 8, 2021.
Frontiers Patch 3.66 - September 13, 2021.
Frontiers Patch 3.67 - September 16, 2021.
Frontiers Patch 3.68 - September 30, 2021.
Emergence 3.70 - October 20, 2021.
Emergence Patch 3.71 - October 28, 2021.
Expeditions Revisited 3.73 - November 24, 2021.
Expeditions Revisited 3.74 - November 26, 2021.
Expeditions Revisited 3.75 - December 3, 2021. 
Sentinel 3.8
Sentinel 3.81 - February 18, 2022.
Sentinel 3.82 - February 22, 2022.
Sentinel 3.84 - March 1, 2022.
Outlaws 3.85
Outlaws 3.87 - April 19, 2022.
Outlaws 3.88 - April 22, 2022.
Outlaws 3.89 - May 3, 2022.
Leviathan 3.90 - May 25, 2022.
Leviathan 3.91 - May 31, 2022.
Leviathan 3.92 - June 13, 2022.
Leviathan 3.93 - June 17, 2022.
Endurance 3.94
Endurance 3.95 - July 22, 2022.
Endurance 3.96 - July 22, 2022.
Endurance 3.97 - July 27, 2022.
Endurance 3.98 - July 31, 2022.
Endurance 3.99 - August 19, 2022.
WayPoint 4.0 
Waypoint 4.0.3 - October 20, 2022.
Waypoint 4.03 - October 8, 2022.
Waypoint 4.04 - October 15, 2022.
Waypoint 4.05 - October 21, 2022.
Waypoint 4.06 - November 8, 2022.
Waypoint Holiday 4.07 - November 23, 2022.
Waypoint Holiday 4.08 - December 6, 2022.
Fractal 4.1
Fractal 4.12 - March 1, 2023.
Fractal 4.13 - March 8, 2023.
Fractal 4.14 - March 10, 2023.
Fractal 4.15 - March 16, 2023.
Interceptor 4.2
Interceptor Patch 4.21 - April 6, 2023.
Interceptor Patch 4.22 - April 14, 2023.
Interceptor Patch 4.23 - April 17, 2023.
Singularity 4.30 - June 7, 2023.
Singularity Patch 4.30.1 - June 08, 2023.
Singularity Patch 4.30.2 - June 8, 2023.
Singularity Patch 4.30.3 - June 9, 2023.
Singularity Patch 4.33 - June 13, 2023.
Singularity Patch 4.34 - June 16, 2023.
Singularity Patch 4.35 - June 17, 2023.
Singularity Patch 4.36 - June 19, 2023.
Singularity Patch 4.37 - July 13, 2023.
Singularity Patch 4.38 - July 20, 2023.
Echoes 4.4
Echoes Patch 4.41 - August 25, 2023.
Echoes Patch 4.42 - August 29, 2023.
Echoes Patch 4.43 - August 30, 2023.
Echoes Patch 4.44 - September 7, 2023.
Echoes Patch 4.45 - September 18, 2023.
Holiday 2023 Expeditions 4.46 - December 8, 2023.
Holiday Expeditions 2023 4.46.2 - December 13, 2023.
Holiday Expeditions 2023 4.47 - December 14, 2023.
Holiday Expeditions 2023 4.48 - January 3, 2024
Omega 4.5
Omega Patch 4.51 - February 15, 2024
Omega Patch 4.52 - February 27, 2024.
Orbital 4.6
Orbitals Patch 4.61 - March 27, 2024.
Orbitals Patch 4.62 - March 27, 2024.
Orbitals Patch 4.63 - March 29, 2024.
Orbitals Patch 4.64 - April 9, 2024.
Orbitals Patch 4.65 - May 1, 2024.
Adrift 4.7 - May 29, 2024.
Adrift Patch 4.71 - May 31, 2024.
Adrift Patch 4.72 June 4, 2024.
Worlds Part 1 5.0 
Worlds Part 1 Patch 5.00.1 - July 19, 2024.
Worlds Part 1 Patch 5.01 - July 22, 2024.
Worlds Part 1 Patch 5.01.1 - July 23, 2024.
Worlds Part 1 Patch 5.02 - July 25, 2024.
Worlds Part 1 Patch 5.03 - July 29, 2024.
"""

# Process raw data
raw_df = process_raw_data(raw_data)

print("Processed raw data:")
print(raw_df.head())
print(raw_df.info())

Processed raw data:
  Version      Name  Date
0    1.04  PC Patch  None
1    1.05  PC Patch  None
2    1.06  PC Patch  None
3    1.07  PC Patch  None
4    1.08  PC Patch  None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Version  195 non-null    object
 1   Name     195 non-null    object
 2   Date     158 non-null    object
dtypes: object(3)
memory usage: 4.7+ KB
None


In [5]:
# Convert Date to datetime and handle NaT values
noman_update_data['Date'] = pd.to_datetime(noman_update_data['Date'], errors='coerce')
raw_df['Date'] = pd.to_datetime(raw_df['Date'], errors='coerce')

# Combine dataframes
combined_df = pd.concat([raw_df, noman_update_data], ignore_index=True)

# Remove duplicates, prioritizing raw data
combined_df = combined_df.sort_values('Date', ascending=False, na_position='last').drop_duplicates(subset='Version', keep='first')

# Sort the dataframe by date
combined_df = combined_df.sort_values('Date')

# Fill NaT values with a date before our analysis period
combined_df['Date'] = combined_df['Date'].fillna(pd.Timestamp('2016-01-01'))

# Filter data from 2019 to June 2024
df = combined_df[(combined_df['Date'] >= '2019-01-01') & (combined_df['Date'] <= '2024-06-30')]

print("Combined and cleaned data:")
print(df.head())
print(df.info())

Combined and cleaned data:
   Version          Name       Date Category Title
28    2.04  Beyond Patch 2019-08-15      NaN   NaN
27    2.06  Beyond Patch 2019-08-15      NaN   NaN
26    2.05  Beyond Patch 2019-08-15      NaN   NaN
29    2.07  Beyond Patch 2019-08-20      NaN   NaN
30    2.08  Beyond Patch 2019-08-21      NaN   NaN
<class 'pandas.core.frame.DataFrame'>
Index: 126 entries, 28 to 152
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Version   126 non-null    object        
 1   Name      126 non-null    object        
 2   Date      126 non-null    datetime64[ns]
 3   Category  0 non-null      object        
 4   Title     0 non-null      object        
dtypes: datetime64[ns](1), object(4)
memory usage: 5.9+ KB
None


In [7]:
# Define major updates
major_updates = [
    "Beyond", "Synthesis", "Living Ship", "Exo Mech", "Desolation", "Origins", 
    "Companions", "Expeditions", "Prisms", "Frontiers", "Sentinel", "Outlaws", 
    "Endurance", "Waypoint", "Fractal", "Interceptor", "Singularity", "Echoes", 
    "Omega", "Orbital", "Adrift", "Worlds", "Holiday", "Cross Play", "Crossplay",
    "Halloween", "Next Generation", "Emergence", "Leviathan", "Beachhead"
]

def categorize_update(name):
    for update in major_updates:
        if update.lower() in str(name).lower():
            return update
    return "Other"

# Use .loc to add the new column to the original DataFrame
df.loc[:, 'Major_Update'] = df['Name'].apply(categorize_update)

print("Updates categorized:")
print(df['Major_Update'].value_counts())

Updates categorized:
Major_Update
Beyond             16
Expeditions        13
Singularity        10
Frontiers           8
Waypoint            7
Orbital             5
Echoes              5
Endurance           5
Synthesis           5
Crossplay           4
Living Ship         4
Fractal             4
Leviathan           4
Sentinel            3
Interceptor         3
Outlaws             3
Adrift              3
Prisms              3
Beachhead           3
Next Generation     3
Origins             3
Emergence           2
Exo Mech            2
Companions          2
Omega               2
Desolation          2
Halloween           1
Cross Play          1
Name: count, dtype: int64
